In [84]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from d2l import tensorflow as d2l
import random

In [85]:
class LinearRegression(d2l.Module): #@save
    """The linear regression model implemented with high-level APIs."""
    """Do d2l"""
    def __init__(self, lr):
        super().__init__()
        self.save_hyperparameters()
        initializer = tf.initializers.RandomNormal(stddev=0.01)
        self.net = tf.keras.layers.Dense(1, kernel_initializer=initializer)

In [86]:
@d2l.add_to_class(LinearRegression)  #@save
def forward(self, X):
    return self.net(X)

In [87]:
class LinearRegressionData(d2l.DataModule): #@save
    """Data for linear regression."""
    def __init__(self, nInputs,data,num_train=5197, num_val=1300, batch_size=32):
        super().__init__()
        self.save_hyperparameters()
        self.X=data[:,0:nInputs]
        self.y=data[:,nInputs]

In [88]:
@d2l.add_to_class(LinearRegression)  #@save
def loss(self, y_hat, y):
    fn = tf.keras.losses.MeanSquaredError()
    return fn(y, y_hat)

In [89]:
@d2l.add_to_class(LinearRegression)  #@save
def configure_optimizers(self):
    return tf.keras.optimizers.SGD(self.lr)

In [90]:
@d2l.add_to_class(d2l.DataModule) #@save
def get_tensorloader(self, tensors, train, indices=slice(0, None)):
    tensors = tuple(a[indices] for a in tensors)
    shuffle_buffer = tensors[0].shape[0] if train else 1
    return tf.data.Dataset.from_tensor_slices(tensors).shuffle(
    buffer_size=shuffle_buffer).batch(self.batch_size)
    
@d2l.add_to_class(LinearRegressionData) #@save
def get_dataloader(self, train):
    i = slice(0, self.num_train) if train else slice(self.num_train, None)
    return self.get_tensorloader((self.X, self.y), train, i)

Preparando os dados:

In [101]:
data = pd.read_csv('winequalityN.csv')
data = pd.get_dummies(data)
for key in data.keys():
    data[key] = data[key].fillna(data[key].mean())
tf.convert_to_tensor(data, dtype=tf.float32)
train_data, val_data = train_test_split(data, test_size=0.2)
train_data


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type_red,type_white
1449,6.9,0.19,0.49,6.6,0.036,49.0,172.0,0.99320,3.20,0.27,11.5,6,False,True
3548,6.2,0.47,0.19,8.3,0.029,24.0,142.0,0.99200,3.22,0.45,12.3,6,False,True
4541,6.0,0.23,0.34,1.3,0.025,23.0,111.0,0.98961,3.36,0.37,12.7,6,False,True
2062,6.9,0.17,0.25,1.6,0.047,34.0,132.0,0.99140,3.16,0.48,11.4,5,False,True
6322,8.3,0.26,0.37,1.4,0.076,8.0,23.0,0.99740,3.26,0.70,9.6,6,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,7.4,0.26,0.31,7.6,0.047,52.0,177.0,0.99620,3.13,0.45,8.9,6,False,True
5549,9.8,0.88,0.25,2.5,0.104,35.0,155.0,1.00100,3.41,0.67,11.2,5,True,False
2276,6.7,0.31,0.18,7.7,0.043,57.0,200.0,0.99566,3.17,0.44,9.4,6,False,True
4746,6.3,0.23,0.50,10.4,0.043,61.0,132.0,0.99542,2.86,0.46,9.1,6,False,True


In [104]:
model = d2l.LinearRegression(lr=0.03)
data = LinearRegressionData(data)
trainer = d2l.Trainer(max_epochs=3)
trainer.fit(model, train_data, data)

TypeError: LinearRegressionData.__init__() missing 1 required positional argument: 'data'